In [15]:
import numpy as np
import pandas as pd
import random
from scipy.io import arff
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_validate
from math import sqrt

In [16]:
def compute_eer_from_paper(labels, prediction):
    from sklearn.metrics import roc_curve
    fprs, tprs, _ = roc_curve(labels, prediction)
    eer = fprs[np.nanargmin(np.absolute((1 - tprs) - fprs))]
    print(eer)
    return eer

In [17]:
def compute_eer_changjiang(labels, prediction):
    from scipy.optimize import brentq
    from scipy.interpolate import interp1d
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(labels, prediction)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

In [18]:
def calculate_eer(y_true, y_score):
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=True)
    abs_diff = [abs(fp-tp) for fp, tp in zip(fpr, tpr)]
    min_idx = abs_diff.index(min(abs_diff))
    eer = (fpr[min_idx] + tpr[min_idx])/2
    print(abs_diff, eer)
    return eer

In [19]:
columns_to_be_dropped = []
for n in ['X', 'Y', 'Z']:
    columns_to_be_dropped.append(f'"{n}VAR"')
    for i in range(13):
        columns_to_be_dropped.append(f'"{n}MFCC{i}"')
for n in combinations(['X', 'Y', 'Z'], 2):
    columns_to_be_dropped.append(f'"{"".join(n)}COS"')
    columns_to_be_dropped.append(f'"{"".join(n)}COR"')

sensors = ["phone_accel", "watch_accel", "phone_gyro", "watch_gyro"]
activities = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
              'I', 'J', 'K', 'L', 'M', 'O', 'P', 'Q', 'R', 'S']

In [20]:
def processed_dataframe(raw: pd.DataFrame, cols_to_drop: list[str]) -> pd.DataFrame:
    result = raw.drop(columns=cols_to_drop)
    result.columns = [col.replace('"', "") for col in result]
    result['ACTIVITY'] = result['ACTIVITY'].str.decode('utf-8')
    result['class'] = result['class'].str.decode('utf-8')
    return result

In [21]:
def pick_impostors(guid: str, people: list[dict[str, pd.DataFrame]]) -> list[dict[str, pd.DataFrame]]:
    return random.choices([i for i in people if i['guid'] != guid], k=18)


def biometric_train_test_split(person, impostors, activity, sensors):
    full_person = person[sensors[0]].query(f"`ACTIVITY`=='{activity}'").drop(
        columns=['ACTIVITY', 'class']).add_suffix(sensors[0])
    full_set_impostors = [impostors[i][sensors[0]].query(f"`ACTIVITY`=='{activity}'").drop(
        columns=['ACTIVITY', 'class'])[:3].add_suffix(sensors[0]) for i in range(len(impostors))]

    for i in range(1, len(sensors)):
        full_person = full_person.merge(person[sensors[i]].query(f"`ACTIVITY`=='{activity}'").drop(
            columns=['ACTIVITY', 'class']).add_suffix(sensors[i]), left_index=True, right_index=True)
        for j in range(len(impostors)):
            full_set_impostors[j] = full_set_impostors[j].merge(impostors[j][sensors[i]].query(f"`ACTIVITY`=='{activity}'").drop(
                columns=['ACTIVITY', 'class'])[:3].add_suffix(sensors[i]), left_index=True, right_index=True)

    full_person['class'] = True
    for i in range(len(full_set_impostors)):
        full_set_impostors[i]['class'] = False

    training_set = pd.concat([full_person[:len(
        full_person)//2]] + full_set_impostors[:len(full_set_impostors)//2])
    testing_set = pd.concat([full_person[len(full_person)//2:]] +
                            full_set_impostors[len(full_set_impostors)//2:])

    return training_set, testing_set

In [29]:
def person_classification_df_builder(people: list[dict[str, int | pd.DataFrame]], sensors: list[str]) -> pd.DataFrame:
    result_df: pd.DataFrame = None
    for person in people:
        df: pd.DataFrame = None
        for sensor in sensors:
            df = person[sensor].drop(columns=['class']).add_suffix(sensor) if df is None else df.merge(
                person[sensor].drop(columns=['class']).add_suffix(sensor), left_index=True, right_index=True)
            df = pd.get_dummies(df)
        df['class']=person['guid']
        result_df = df if result_df is None else pd.concat([result_df, df])
    return result_df.dropna()

def activity_classification_df_builder(people: list[dict[str, int | pd.DataFrame]], sensors: list[str]) -> pd.DataFrame:
    result_df: pd.DataFrame = None
    for person in people:
        df: pd.DataFrame = None
        for sensor in sensors:
            df = person[sensor].drop(columns=['class','ACTIVITY']).add_suffix(sensor) if df is None else df.merge(
                person[sensor].drop(columns=['class','ACTIVITY']).add_suffix(sensor), left_index=True, right_index=True)
        result_df = df if result_df is None else pd.concat([result_df, df])
    return result_df.dropna()

In [23]:
people: list[dict[str, pd.DataFrame]] = [{
    'guid': i,
    'phone_accel': processed_dataframe(pd.DataFrame(arff.loadarff(f"arff_files/phone/accel/data_{i}_accel_phone.arff")[0]), columns_to_be_dropped),
    'phone_gyro': processed_dataframe(pd.DataFrame(arff.loadarff(f"arff_files/phone/gyro/data_{i}_gyro_phone.arff")[0]), columns_to_be_dropped),
    'watch_accel': processed_dataframe(pd.DataFrame(arff.loadarff(f"arff_files/watch/accel/data_{i}_accel_watch.arff")[0]), columns_to_be_dropped),
    'watch_gyro': processed_dataframe(pd.DataFrame(arff.loadarff(f"arff_files/watch/gyro/data_{i}_gyro_watch.arff")[0]), columns_to_be_dropped)
} for i in range(1600, 1651) if i != 1614]  # person 1614 missing in files

In [24]:
every_combination = [[i]for i in sensors] + \
    [list(i) for i in list(combinations(sensors, 2))]+[sensors]

### Authentication

In [25]:
# biometry_models = {person['guid']:{activity: {'+'.join(combination): tuple[RandomForestClassifier,float] for combination in every_combination} for activity in activities } for person in people}
# for person in people:
#     impostors = pick_impostors(person['guid'], people)
#     for activity in activities:
#         for combination in every_combination:
#             train, test = biometric_train_test_split(
#                 person, impostors, activity, combination)
#             if len(train) == 0 or len(test)==0:
#                 continue
#             train_target = train['class']
#             train = train.drop(columns=['class'])
#             test_target = test['class']
#             test = test.drop(columns=['class'])
#             biometry_classifier = RandomForestClassifier(
#                 10, max_features=int(sqrt(len(train_target.columns))))
#             classifier.fit(train, train_target)
#             biometry_models[person['guid']][activity]['+'.join(combination)] = (biometry_classifier, compute_eer_from_paper(test_target,biometry_classifier.predict(test)))

### People classification

In [26]:
# people_models = {person['guid']: {'+'.join(combination): tuple[RandomForestClassifier, float]
#                                   for combination in every_combination} for person in people}
# kf = KFold(n_splits=10)
# people_df = person_classification_df_builder(people, sensors)
# people_target = people_df['class']
# people_df = people_df.drop(columns=['class'])
# people_classifier = RandomForestClassifier(10, max_features=int(sqrt(len(people_df.columns))))
# # people_classifier.fit(people_df,people_target)
# score = cross_validate(people_classifier,people_df,people_target,cv=kf,scoring='accuracy')
# score